In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
def selectglaciersrgitable(glac_no=None,
                           rgi_regionsO1=None,
                           rgi_regionsO2=None,
                           rgi_glac_number=None,
#                            rgi_fp=input.rgi_fp,
                           rgi_fp = '/Users/davidrounce/Documents/Dave_Rounce/HiMAT/RGI/rgi60/00_rgi60_attribs/',
                           rgi_cols_drop=['GLIMSId','BgnDate','EndDate','Status','Connect','Linkages','Name'],
                           rgi_O1Id_colname='glacno',
                           rgi_glacno_float_colname='RGIId_float',
                           indexname='GlacNo'):
    """
    Select all glaciers to be used in the model run according to the regions and glacier numbers defined by the RGI
    glacier inventory. This function returns the rgi table associated with all of these glaciers.

    glac_no : list of strings
        list of strings of RGI glacier numbers (e.g., ['1.00001', '13.00001'])
    rgi_regionsO1 : list of integers
        list of integers of RGI order 1 regions (e.g., [1, 13])
    rgi_regionsO2 : list of integers or 'all'
        list of integers of RGI order 2 regions or simply 'all' for all the order 2 regions
    rgi_glac_number : list of strings
        list of RGI glacier numbers without the region (e.g., ['00001', '00002'])

    Output: Pandas DataFrame of the glacier statistics for each glacier in the model run
    (rows = GlacNo, columns = glacier statistics)
    """
    if glac_no is not None:
        glac_no_byregion = {}
        rgi_regionsO1 = [int(i.split('.')[0]) for i in glac_no]
        rgi_regionsO1 = list(set(rgi_regionsO1))
        for region in rgi_regionsO1:
            glac_no_byregion[region] = []
        for i in glac_no:
            region = i.split('.')[0]
            glac_no_only = i.split('.')[1]
            glac_no_byregion[int(region)].append(glac_no_only)

        for region in rgi_regionsO1:
            glac_no_byregion[region] = sorted(glac_no_byregion[region])

    # Create an empty dataframe
    rgi_regionsO1 = sorted(rgi_regionsO1)
    glacier_table = pd.DataFrame()
    for region in rgi_regionsO1:

        if glac_no is not None:
            rgi_glac_number = glac_no_byregion[region]

#        if len(rgi_glac_number) < 50:

        for i in os.listdir(rgi_fp):
            if i.startswith(str(region).zfill(2)) and i.endswith('.csv'):
                rgi_fn = i
        try:
            csv_regionO1 = pd.read_csv(rgi_fp + rgi_fn)
        except:
            csv_regionO1 = pd.read_csv(rgi_fp + rgi_fn, encoding='latin1')
        
        # Populate glacer_table with the glaciers of interest
        if rgi_regionsO2 == 'all' and rgi_glac_number == 'all':
            print("All glaciers within region(s) %s are included in this model run." % (region))
            if glacier_table.empty:
                glacier_table = csv_regionO1
            else:
                glacier_table = pd.concat([glacier_table, csv_regionO1], axis=0)
        elif rgi_regionsO2 != 'all' and rgi_glac_number == 'all':
            print("All glaciers within subregion(s) %s in region %s are included in this model run." %
                  (rgi_regionsO2, region))
            for regionO2 in rgi_regionsO2:
                if glacier_table.empty:
                    glacier_table = csv_regionO1.loc[csv_regionO1['O2Region'] == regionO2]
                else:
                    glacier_table = (pd.concat([glacier_table, csv_regionO1.loc[csv_regionO1['O2Region'] ==
                                                                                regionO2]], axis=0))
        else:
            if len(rgi_glac_number) < 20:
                print("%s glaciers in region %s are included in this model run: %s" % (len(rgi_glac_number), region,
                                                                                       rgi_glac_number))
            else:
                print("%s glaciers in region %s are included in this model run: %s and more" %
                      (len(rgi_glac_number), region, rgi_glac_number[0:50]))
                
            rgiid_subset = ['RGI60-' + str(region).zfill(2) + '.' + x for x in rgi_glac_number] 
            rgiid_all = list(csv_regionO1.RGIId.values)
            rgi_idx = [rgiid_all.index(x) for x in rgiid_subset]
            if glacier_table.empty:
                glacier_table = csv_regionO1.loc[rgi_idx]
            else:
                glacier_table = (pd.concat([glacier_table, csv_regionO1.loc[rgi_idx]],
                                           axis=0))
                    
    glacier_table = glacier_table.copy()
    # reset the index so that it is in sequential order (0, 1, 2, etc.)
    glacier_table.reset_index(inplace=True)
    # change old index to 'O1Index' to be easier to recall what it is
    glacier_table.rename(columns={'index': 'O1Index'}, inplace=True)
    # Record the reference date
    glacier_table['RefDate'] = glacier_table['BgnDate']
    # if there is an end date, then roughly average the year
    enddate_idx = glacier_table.loc[(glacier_table['EndDate'] > 0), 'EndDate'].index.values
    glacier_table.loc[enddate_idx,'RefDate'] = (
            np.mean((glacier_table.loc[enddate_idx,['BgnDate', 'EndDate']].values / 10**4).astype(int),
                    axis=1).astype(int) * 10**4 + 9999)
    # drop columns of data that is not being used
    glacier_table.drop(rgi_cols_drop, axis=1, inplace=True)
    # add column with the O1 glacier numbers
    glacier_table[rgi_O1Id_colname] = (
            glacier_table['RGIId'].str.split('.').apply(pd.Series).loc[:,1].astype(int))
    glacier_table['rgino_str'] = [x.split('-')[1] for x in glacier_table.RGIId.values]
    glacier_table[rgi_glacno_float_colname] = (np.array([np.str.split(glacier_table['RGIId'][x],'-')[1]
                                                    for x in range(glacier_table.shape[0])]).astype(float))
    # set index name
    glacier_table.index.name = indexname

    print("This study is focusing on %s glaciers in region %s" % (glacier_table.shape[0], rgi_regionsO1))

    return glacier_table


def nearest_nonzero_idx(a,x,y):
    r,c = np.nonzero(a)
    min_idx = ((r - x)**2 + (c - y)**2).argmin()
    return r[min_idx], c[min_idx]


def emergence_pixels(gf, vel_x_raw, vel_y_raw, icethickness_raw, xres, yres, 
                     vel_min=0, max_velocity=600, vel_depth_avg_factor=0.8, option_border=1,
                     positive_is_east=True, positive_is_north=True, constant_icethickness=False, debug=True):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Glacier mask
    glac_mask = np.zeros(vel_x_raw.shape) + 1
    glac_mask[gf.z1.mask] = 0
    
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    if positive_is_north:
        vel_y = -1*vel_y_raw * vel_depth_avg_factor
    else:
        vel_y = vel_y_raw * vel_depth_avg_factor
    if positive_is_east:
        vel_x = vel_x_raw * vel_depth_avg_factor
    else:
        vel_x = -1*vel_x_raw * vel_depth_avg_factor
    vel_total = (vel_y**2 + vel_x**2)**0.5
    # Ice thickness
    icethickness = icethickness_raw.copy()
    if constant_icethickness:
        icethickness[:,:] = 1
        icethickness = icethickness * glac_mask
#     print('mean ice thickness:', np.round(icethickness.mean(),0), 'm')
    # Compute the initial volume
    volume_initial = icethickness * (xres * yres)
    pix_maxres = xres
    if yres > pix_maxres:
        pix_maxres = yres
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pix_maxres) + 1
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    vel_x[vel_total < vel_min] = 0
    vel_y[vel_total < vel_min] = 0
    vel_x[vel_total > max_velocity] = 0
    vel_y[vel_total > max_velocity] = 0
#     # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
#     if option_stagnantbands == 1:
#         vel_x[bands <= stagnant_band] = 0
#         vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / xres
    vel_y_pix = vel_y / yres
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
          
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
         
    # Redistribute off-glacier volume back onto the nearest pixel on the glacier
    offglac_row, offglac_col = np.where((glac_mask == 0) & (volume_final > 0))
    for nidx in range(0,len(offglac_row)):
        nrow = offglac_row[nidx]
        ncol = offglac_col[nidx]
        ridx, cidx = nearest_nonzero_idx(glac_mask, nrow, ncol)
        # Add off-glacier volume back onto nearest pixel on glacier
        volume_final[ridx,cidx] += volume_final[nrow,ncol]
        volume_final[nrow,ncol] = 0
            
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    if debug:
        print('Mass is conserved?', np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() < 0.01)
        print(np.round(np.absolute(volume_final.sum() - volume_initial.sum()),1), 
              np.round(np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() * 100,2), '%')
        
    if np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() > 0.01:
        print('MASS NOT CONSERVED FOR EMERGENCE VELOCITY')
    # Final ice thickness
    icethickness_final = volume_final / (xres * yres)
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity



class GlacFeat:
    def __init__(self, feat, glacname_fieldname, glacnum_fieldname):

        self.glacname = feat.GetField(glacname_fieldname)
        if self.glacname is None:
            self.glacname = ""
        else:
            #RGI has some nonstandard characters
            #self.glacname = self.glacname.decode('unicode_escape').encode('ascii','ignore')
            #glacname = re.sub(r'[^\x00-\x7f]',r'', glacname)
            self.glacname = re.sub(r'\W+', '', self.glacname)
            self.glacname = self.glacname.replace(" ", "")
            self.glacname = self.glacname.replace("_", "")
            self.glacname = self.glacname.replace("/", "")

        self.glacnum = feat.GetField(glacnum_fieldname)
        fn = feat.GetDefnRef().GetName()
        #RGIId (String) = RGI50-01.00004
        self.glacnum = '%0.5f' % float(self.glacnum.split('-')[-1])

        if self.glacname:
            self.feat_fn = "%s_%s" % (self.glacnum, self.glacname)
        else:
            self.feat_fn = str(self.glacnum)

        self.glac_geom_orig = geolib.geom_dup(feat.GetGeometryRef())
        self.glac_geom = geolib.geom_dup(self.glac_geom_orig)
        #Hack to deal with fact that this is not preserved in geom when loaded from pickle on disk
        self.glac_geom_srs_wkt = self.glac_geom.GetSpatialReference().ExportToWkt()

        #Attributes written by mb_calc
        self.z1 = None
        self.z1_hs = None
        self.z1_stats = None
        self.z1_ela = None
        self.z2 = None
        self.z2_hs = None
        self.z2_stats = None
        self.z2_ela = None
        self.z2_aspect = None
        self.z2_aspect_stats = None
        self.z2_slope = None
        self.z2_slope_stats = None
        self.res = None
        self.dhdt = None
        self.mb = None
        self.mb_mean = None
        self.t1 = None
        self.t2 = None
        self.dt = None
        self.t1_mean = None
        self.t2_mean = None
        self.dt_mean = None

        self.H = None
        self.H_mean = np.nan
        self.vx = None
        self.vy = None
        self.vm = None
        self.vm_mean = np.nan
        self.divQ = None
        self.emvel = None
        self.debris_class = None
        self.debris_thick = None
        self.debris_thick_mean = np.nan
        self.perc_clean = np.nan
        self.perc_debris = np.nan
        self.perc_pond = np.nan

    def geom_srs_update(self, srs=None):
        if self.glac_geom.GetSpatialReference() is None:
            if srs is None:
                srs = osr.SpatialReference()
                srs.ImportFromWkt(self.glac_geom_srs_wkt)
            self.glac_geom.AssignSpatialReference(srs)

    def geom_attributes(self, srs=None):
        self.geom_srs_update()
        if srs is not None:
            #Should reproject here to equal area, before geom_attributes
            #self.glac_geom.AssignSpatialReference(glac_shp_srs)
            #self.glac_geom_local = geolib.geom2localortho(self.glac_geom)
            geolib.geom_transform(self.glac_geom, srs)

        self.glac_geom_extent = geolib.geom_extent(self.glac_geom)
        self.glac_area = self.glac_geom.GetArea()
        self.glac_area_km2 = self.glac_area / 1E6
        self.cx, self.cy = self.glac_geom.Centroid().GetPoint_2D()
        
        
#RGI uses 50 m bins
def hist_plot(gf, bin_width=50.0, dz_clim=(-2.0, 2.0), exportcsv=True, csv_ending=''):
    #print("Generating histograms")
    #Create bins for full range of input data and specified bin width

    #NOTE: these counts/areas are for valid pixels only
    #Not necessarily a true representation of actual glacier hypsometry
    #Need a void-filled DEM for this

    z_bin_edges, z_bin_centers = malib.get_bins(gf.z1, bin_width)
    #Need to compress here, otherwise histogram uses masked values!
    z1_bin_counts, z1_bin_edges = np.histogram(gf.z1.compressed(), bins=z_bin_edges)
    z1_bin_areas = z1_bin_counts * gf.res[0] * gf.res[1] / 1E6
    #RGI standard is integer thousandths of glaciers total area
    #Should check to make sure sum of bin areas equals total area
    #z1_bin_areas_perc = 100. * z1_bin_areas / np.sum(z1_bin_areas)
    z1_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)

    #If we only have one elevation grid with dhdt
    if gf.z2 is not None:
        z2_bin_counts, z2_bin_edges = np.histogram(gf.z2.compressed(), bins=z_bin_edges)
        z2_bin_areas = z2_bin_counts * gf.res[0] * gf.res[1] / 1E6
        #z2_bin_areas_perc = 100. * z2_bin_areas / np.sum(z2_bin_areas)
        z2_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)
    else:
        z2_bin_counts = z1_bin_counts
        z2_bin_edges = z1_bin_edges
        z2_bin_areas = z1_bin_areas
        z2_bin_areas_perc = z1_bin_areas_perc

    #Create arrays to store output
    slope_bin_med = np.ma.masked_all_like(z1_bin_areas)
    slope_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_med = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.dhdt is not None:
        mb_bin_med = np.ma.masked_all_like(z1_bin_areas)
        np.ma.set_fill_value(mb_bin_med, np.nan)
        mb_bin_mad = np.ma.masked_all_like(mb_bin_med)
        mb_bin_mean = np.ma.masked_all_like(mb_bin_med)
        mb_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_med = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mad = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mean = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_count = np.ma.masked_all_like(mb_bin_med)
    if gf.vm is not None:
        vm_bin_med = np.ma.masked_all_like(z1_bin_areas)
        vm_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.H is not None:
        H_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        H_bin_std = np.ma.masked_all_like(z1_bin_areas)
    if gf.emvel is not None:
        emvel_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_std = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_med = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.debris_class is not None:
#         perc_clean = np.ma.masked_all_like(z1_bin_areas)
#         perc_debris = np.ma.masked_all_like(z1_bin_areas)
#         perc_pond = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_mad = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_clean_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_debris_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_pond_bin_med = np.ma.masked_all_like(mz1_bin_areas)

#         gf.dhdt_clean = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 1).data))
#         gf.dhdt_debris = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 2).data))
#         gf.dhdt_pond = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 3).data))

    if gf.debris_thick_ts is not None:
        debris_thick_ts_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_ts_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.meltfactor_ts is not None:
        meltfactor_ts_med = np.ma.masked_all_like(z1_bin_areas)
        meltfactor_ts_mad = np.ma.masked_all_like(z1_bin_areas)

    #Bin sample count must be greater than this value
    min_bin_samp_count = 9

    #Loop through each bin and extract stats
    idx = np.digitize(gf.z1, z_bin_edges)
    for bin_n in range(z_bin_centers.size):
        if gf.dhdt is not None:
            mb_bin_samp = gf.mb_map[(idx == bin_n+1)]
            if mb_bin_samp.count() > min_bin_samp_count:
                mb_bin_med[bin_n] = malib.fast_median(mb_bin_samp)
                mb_bin_mad[bin_n] = malib.mad(mb_bin_samp)
                mb_bin_mean[bin_n] = mb_bin_samp.mean()
                mb_bin_std[bin_n] = mb_bin_samp.std()
            dhdt_bin_samp = gf.dhdt[(idx == bin_n+1)]
            if dhdt_bin_samp.count() > min_bin_samp_count:
                dhdt_bin_med[bin_n] = malib.fast_median(dhdt_bin_samp)
                dhdt_bin_mad[bin_n] = malib.mad(dhdt_bin_samp)
                dhdt_bin_mean[bin_n] = dhdt_bin_samp.mean()
                dhdt_bin_std[bin_n] = dhdt_bin_samp.std()
                dhdt_bin_count[bin_n] = dhdt_bin_samp.count()
        if gf.debris_thick is not None:
            debris_thick_bin_samp = gf.debris_thick[(idx == bin_n+1)]
            if debris_thick_bin_samp.size > min_bin_samp_count:
                debris_thick_med[bin_n] = malib.fast_median(debris_thick_bin_samp)
                debris_thick_mad[bin_n] = malib.mad(debris_thick_bin_samp)
        
        if gf.debris_thick_ts is not None:
            debris_thick_ts_bin_samp = gf.debris_thick_ts[(idx == bin_n+1)]
            if debris_thick_ts_bin_samp.size > min_bin_samp_count:
                debris_thick_ts_med[bin_n] = malib.fast_median(debris_thick_ts_bin_samp)
                debris_thick_ts_mad[bin_n] = malib.mad(debris_thick_ts_bin_samp)
        if gf.meltfactor_ts is not None:
            meltfactor_ts_bin_samp = gf.meltfactor_ts[(idx == bin_n+1)]
            if meltfactor_ts_bin_samp.size > min_bin_samp_count:
                meltfactor_ts_med[bin_n] = malib.fast_median(meltfactor_ts_bin_samp)
                meltfactor_ts_mad[bin_n] = malib.mad(meltfactor_ts_bin_samp)
                
        if gf.debris_class is not None:
            debris_class_bin_samp = gf.debris_class[(idx == bin_n+1)]
            dhdt_clean_bin_samp = gf.dhdt_clean[(idx == bin_n+1)]
            dhdt_debris_bin_samp = gf.dhdt_debris[(idx == bin_n+1)]
            dhdt_pond_bin_samp = gf.dhdt_pond[(idx == bin_n+1)]
            if debris_class_bin_samp.count() > min_bin_samp_count:
                perc_clean[bin_n] = 100. * (debris_class_bin_samp == 1).sum()/debris_class_bin_samp.count()
                perc_debris[bin_n] = 100. * (debris_class_bin_samp == 2).sum()/debris_class_bin_samp.count()
                perc_pond[bin_n] = 100. * (debris_class_bin_samp == 3).sum()/debris_class_bin_samp.count()
            if dhdt_clean_bin_samp.count() > min_bin_samp_count:
                dhdt_clean_bin_med[bin_n] = malib.fast_median(dhdt_clean_bin_samp)
            if dhdt_debris_bin_samp.count() > min_bin_samp_count:
                dhdt_debris_bin_med[bin_n] = malib.fast_median(dhdt_debris_bin_samp)
            if dhdt_pond_bin_samp.count() > min_bin_samp_count:
                dhdt_pond_bin_med[bin_n] = malib.fast_median(dhdt_pond_bin_samp)
        if gf.vm is not None:
            vm_bin_samp = gf.vm[(idx == bin_n+1)]
            if vm_bin_samp.size > min_bin_samp_count:
                vm_bin_med[bin_n] = malib.fast_median(vm_bin_samp)
                vm_bin_mad[bin_n] = malib.mad(vm_bin_samp)
        if gf.H is not None:
            H_bin_samp = gf.H[(idx == bin_n+1)]
            if H_bin_samp.size > min_bin_samp_count:
                H_bin_mean[bin_n] = H_bin_samp.mean()
                H_bin_std[bin_n] = H_bin_samp.std()
        if gf.emvel is not None:
            emvel_bin_samp = gf.emvel[(idx == bin_n+1)]
            if emvel_bin_samp.size > min_bin_samp_count:
                emvel_bin_mean[bin_n] = emvel_bin_samp.mean()
                emvel_bin_std[bin_n] = emvel_bin_samp.std()
                emvel_bin_med[bin_n] = malib.fast_median(emvel_bin_samp)
                emvel_bin_mad[bin_n] = malib.mad(emvel_bin_samp)
        slope_bin_samp = gf.z1_slope[(idx == bin_n+1)]
        if slope_bin_samp.size > min_bin_samp_count:
            slope_bin_med[bin_n] = malib.fast_median(slope_bin_samp)
            slope_bin_mad[bin_n] = malib.mad(slope_bin_samp)
        aspect_bin_samp = gf.z1_aspect[(idx == bin_n+1)]
        if aspect_bin_samp.size > min_bin_samp_count:
            aspect_bin_med[bin_n] = malib.fast_median(aspect_bin_samp)
            aspect_bin_mad[bin_n] = malib.mad(aspect_bin_samp)

    if gf.dhdt is not None:
        dhdt_bin_areas = dhdt_bin_count * gf.res[0] * gf.res[1] / 1E6
        #dhdt_bin_areas_perc = 100. * dhdt_bin_areas / np.sum(dhdt_bin_areas)
        dhdt_bin_areas_perc = 100. * (dhdt_bin_areas / gf.glac_area_km2)

    outbins_header = 'bin_center_elev_m, z1_bin_count_valid, z1_bin_area_valid_km2, z1_bin_area_perc, z2_bin_count_valid, z2_bin_area_valid_km2, z2_bin_area_perc, slope_bin_med, aspect_bin_med'
    fmt = '%0.1f, %0.0f, %0.3f, %0.2f, %0.0f, %0.3f, %0.2f, %0.2f, %0.2f'
    outbins = [z_bin_centers, z1_bin_counts, z1_bin_areas, z1_bin_areas_perc, z2_bin_counts, z2_bin_areas, z2_bin_areas_perc, slope_bin_med, aspect_bin_med]
    if gf.dhdt is not None:
        outbins_header += ', dhdt_bin_count, dhdt_bin_area_valid_km2, dhdt_bin_area_perc, dhdt_bin_med_ma, dhdt_bin_mad_ma, dhdt_bin_mean_ma, dhdt_bin_std_ma, mb_bin_med_mwea, mb_bin_mad_mwea, mb_bin_mean_mwea, mb_bin_std_mwea'
        fmt += ', %0.0f, %0.3f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([dhdt_bin_count, dhdt_bin_areas, dhdt_bin_areas_perc, dhdt_bin_med, dhdt_bin_mad, dhdt_bin_mean, dhdt_bin_std, \
                        mb_bin_med, mb_bin_mad, mb_bin_mean, mb_bin_std])
    if gf.debris_thick is not None:
        outbins_header += ', debris_thick_med_m, debris_thick_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_med[debris_thick_med == -(np.inf)] = 0.00
        debris_thick_mad[debris_thick_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_med, debris_thick_mad])
    
    if gf.debris_thick_ts is not None:
        outbins_header += ',debris_thick_ts_med_m,debris_thick_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_ts_med[debris_thick_ts_med == -(np.inf)] = 0.00
        debris_thick_ts_mad[debris_thick_ts_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_ts_med, debris_thick_ts_mad])
    if gf.meltfactor_ts is not None:
        outbins_header += ',meltfactor_ts_med_m,meltfactor_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        meltfactor_ts_med[meltfactor_ts_med == -(np.inf)] = 1
        meltfactor_ts_med[meltfactor_ts_med > 1] = 1
        meltfactor_ts_med[meltfactor_ts_med <= 0] = 1
        meltfactor_ts_mad[meltfactor_ts_mad == -(np.inf)] = 0
        meltfactor_ts_mad[meltfactor_ts_mad > 1] = 0
        meltfactor_ts_mad[meltfactor_ts_mad <= 0] = 0
        outbins.extend([meltfactor_ts_med, meltfactor_ts_mad])
    
    if gf.debris_class is not None:
        outbins_header += ', perc_debris, perc_pond, perc_clean, dhdt_debris_med, dhdt_pond_med, dhdt_clean_med'
        fmt += ', %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([perc_debris, perc_pond, perc_clean, dhdt_debris_bin_med, dhdt_pond_bin_med, dhdt_clean_bin_med])
    if gf.vm is not None:
        outbins_header += ', vm_med, vm_mad'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([vm_bin_med, vm_bin_mad])
    if gf.H is not None:
        outbins_header += ', H_mean, H_std'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([H_bin_mean, H_bin_std])
#         outbins_header += ', H_mean, H_std, emvel_mean, emvel_std'
#         fmt += ', %0.2f, %0.2f, %0.2f, %0.2f'
#         outbins.extend([H_bin_mean, H_bin_std, emvel_bin_mean, emvel_bin_std])

    if gf.emvel is not None:
        outbins_header += ', emvel_mean, emvel_std, emvel_med, emvel_mad'
        fmt += ', %0.3f, %0.3f, %0.3f, %0.3f'
        outbins.extend([emvel_bin_mean, emvel_bin_std, emvel_bin_med, emvel_bin_mad])

    outbins = np.ma.array(outbins).T.astype('float32')
    np.ma.set_fill_value(outbins, np.nan)
    outbins = outbins.filled(np.nan)
    if exportcsv:
        outbins_fn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
        np.savetxt(outbins_fn, outbins, fmt=fmt, delimiter=',', header=outbins_header)

#     #Create plots of elevation bins
#     #print("Generating aed plot")
#     #f,axa = plt.subplots(1,2, figsize=(6, 6))
#     nsubplots = 0
#     if gf.dhdt is not None:
#         nsubplots += 1
#     if gf.debris_thick is not None:
#         nsubplots += 1
#     if gf.vm is not None:
#         nsubplots += 1
#     if gf.H is not None:
#         nsubplots += 1
# #     print(nsubplots)
#     f,axa = plt.subplots(1,nsubplots, squeeze=False, figsize=(10, 7.5))
#     f.suptitle(gf.feat_fn)
#     fs = 9
#     nplot = -1
#     if gf.dhdt is not None:
#         nplot += 1
#         axa[0,nplot].plot(z1_bin_areas, z_bin_centers, label='%0.2f' % gf.t1_mean)
#         axa[0,nplot].axhline(gf.z1_ela, ls=':', c='C0')
#         if gf.z2 is not None:
#             axa[0,nplot].plot(z2_bin_areas, z_bin_centers, label='%0.2f' % gf.t2_mean)
#             axa[0,nplot].axhline(gf.z2_ela, ls=':', c='C1')
#         axa[0,nplot].legend(prop={'size':8}, loc='upper right')
#         axa[0,nplot].set_ylabel('Elevation (m WGS84)', fontsize=fs)
#         axa[0,nplot].set_xlabel('Area $\mathregular{km^2}$', fontsize=fs)
#         axa[0,nplot].yaxis.set_ticks_position('both')
#         # pltlib.minorticks_on(axa[0])

#         nplot += 1
#         axa[0,nplot].axvline(0, lw=1.0, c='k')
#         """
#         #Plot flux divergence values for each bin
#         if gf.vm is not None and gf.H is not None:
#             divQ_bin_mean = np.gradient(H_bin_mean * vm_bin_med * v_col_f)
#             axa[1].plot(divQ_bin_mean, z_bin_centers, color='green')
#         """
#         axa[0,nplot].plot(mb_bin_med, z_bin_centers, color='k')
#         axa[0,nplot].axvline(gf.mb_mean, lw=0.5, ls=':', c='k', label='%0.2f m w.e./yr' % gf.mb_mean)
#         axa[0,nplot].fill_betweenx(z_bin_centers, mb_bin_med-mb_bin_mad, mb_bin_med+mb_bin_mad, color='k', alpha=0.1)
#         axa[0,nplot].fill_betweenx(z_bin_centers, 0, mb_bin_med, where=(mb_bin_med<0), color='r', alpha=0.2)
#         axa[0,nplot].fill_betweenx(z_bin_centers, 0, mb_bin_med, where=(mb_bin_med>0), color='b', alpha=0.2)
#         #axa[nplot].set_xlabel('dh/dt (m/yr)')
#         axa[0,nplot].set_xlabel('Mass balance (m w.e./yr)', fontsize=fs)
#         axa[0,nplot].legend(prop={'size':8}, loc='upper right')
#         axa[0,nplot].yaxis.set_ticks_position('both')
#         # pltlib.minorticks_on(axa[1])
#         #Hide y-axis labels
#         axa[0,nplot].axes.yaxis.set_ticklabels([])
#         axa[0,nplot].set_xlim(*dz_clim)

#     if gf.debris_thick is not None:
#         nplot += 1
#         axa[0,nplot].errorbar(debris_thick_med*100., z_bin_centers, xerr=debris_thick_mad*100, color='k', fmt='o', ms=3, label='Debris Thickness', alpha=0.6)
#     if gf.debris_class is not None:
#         axa[0,nplot].plot(perc_debris, z_bin_centers, color='sienna', label='Debris Coverage')
#         axa[0,nplot].plot(perc_pond, z_bin_centers, color='turquoise', label='Pond Coverage')
#     if gf.debris_thick is not None or gf.debris_class is not None:
#         axa[0,nplot].set_xlim(0, 100)
#         axa[0,nplot].yaxis.set_ticks_position('both')
#         # pltlib.minorticks_on(axa[2])
#         axa[0,nplot].axes.yaxis.set_ticklabels([])
#         axa[0,nplot].legend(prop={'size':8}, loc='upper right')
#         axa[0,nplot].set_xlabel('Debris thickness (cm), coverage (%)', fontsize=fs)

#     if gf.H is not None:
#         nplot += 1
#         axa[0,nplot].plot(H_bin_mean, z_bin_centers, color='b', label='H (%0.2f m)' % gf.H_mean)
#         axa[0,nplot].fill_betweenx(z_bin_centers, H_bin_mean-H_bin_std, H_bin_mean+H_bin_std, color='b', alpha=0.1)
#         axa[0,nplot].set_xlabel('Ice Thickness (m)', fontsize=fs)
#         axa[0,nplot].legend(prop={'size':8}, loc='lower right')
#         # pltlib.minorticks_on(axa[3])
#         #axa[nplot].set_xlim(0, 400)
#         axa[0,nplot].yaxis.tick_left()
#         axa[0,nplot].yaxis.set_ticks_position('both')
#         axa[0,nplot].yaxis.set_label_position("right")
    
#     if gf.vm is not None:
#         nplot += 1
# #         ax4 = axa[0,nplot].twinx()
#         axa[0,nplot].set_xlabel('Velocity (m/yr)', fontsize=fs)
#         axa[0,nplot].plot(vm_bin_med, z_bin_centers, color='g', label='Vm (%0.2f m/yr)' % gf.vm_mean)
#         axa[0,nplot].fill_betweenx(z_bin_centers, vm_bin_med-vm_bin_mad, vm_bin_med+vm_bin_mad, color='g', alpha=0.1)
#         #ax4.set_xlim(0, 50)
#         axa[0,nplot].xaxis.tick_bottom()
#         axa[0,nplot].xaxis.set_label_position("bottom")
#         axa[0,nplot].legend(prop={'size':8}, loc='upper right')
        
#         nplot += 1
# #         axa[0,nplot].set_xlabel('divQ (??)', fontsize=fs)
# #         axa[0,nplot].plot(vm_bin_med, z_bin_centers, color='g', label='Vm (%0.2f m/yr)' % gf.vm_mean)
# #         axa[0,nplot].fill_betweenx(z_bin_centers, vm_bin_med-vm_bin_mad, vm_bin_med+vm_bin_mad, color='g', alpha=0.1)
# #         #ax4.set_xlim(0, 50)
# #         axa[0,nplot].xaxis.tick_bottom()
# #         axa[0,nplot].xaxis.set_label_position("bottom")
# #         axa[0,nplot].legend(prop={'size':8}, loc='upper right')
# #         gf.divQ
    
# #     if gf.vm is not None:
# #         nplot += 1
# # #         ax4 = axa[0,nplot].twinx()
# #         axa[0,nplot].set_xlabel('Velocity (m/yr)', fontsize=fs)
# #         axa[0,nplot].plot(vm_bin_med, z_bin_centers, color='g', label='Vm (%0.2f m/yr)' % gf.vm_mean)
# #         axa[0,nplot].fill_betweenx(z_bin_centers, vm_bin_med-vm_bin_mad, vm_bin_med+vm_bin_mad, color='g', alpha=0.1)
# #         #ax4.set_xlim(0, 50)
# #         axa[0,nplot].xaxis.tick_bottom()
# #         axa[0,nplot].xaxis.set_label_position("bottom")
# #         axa[0,nplot].legend(prop={'size':8}, loc='upper right')

#     plt.tight_layout()
#     #Make room for suptitle
#     plt.subplots_adjust(top=0.95, wspace=0.1)
#     #print("Saving aed plot")
#     fig_fn = os.path.join(outdir_fig, gf.feat_fn+'_mb_aed.png')
#     #plt.savefig(fig_fn, bbox_inches='tight', dpi=300)
#     plt.savefig(fig_fn, dpi=300)
#     plt.close(f)
    
    
    outbins_df = pd.DataFrame(outbins, columns=outbins_header.split(','))
    return outbins_df, z_bin_edges
#     return z_bin_edges

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

# import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib

In [3]:
import globaldebris_input as input

#INPUT
# topdir='/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/'
# #Output directory
# outdir = topdir + 'Shean_2019_0213/mb_combined_20190213_nmad_bins/'
# outdir_fig = outdir + '/figures/'
# outdir_csv = outdir + '/csv'

verbose=False
extra_layers=True
min_glac_area_writeout=0
min_valid_area_perc = 0
buff_dist = 1000
bin_width = 5

ts_info_fullfn = input.ts_fp + input.roi + '_debris_tsinfo.nc'

print(ts_info_fullfn)

#INPUT
glac_shp_fn_dict = {'13':input.main_directory + '/../../../HiMAT/RGI/rgi60/13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp',
                    '14':input.main_directory + '/../../../HiMAT/RGI/rgi60/14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp',
                    '15':input.main_directory + '/../../../HiMAT/RGI/rgi60/15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'}
glac_shp_proj_fp = input.output_fp + 'glac_shp_proj/'
if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)

#DEM
z1_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/surface_DEMs_RGI60/' + 
          'surface_DEMs_RGI60-XXXX/')
z1_fn_sample = 'surface_DEM_RGI60-XXXX.tif'
# Ice thickness
huss_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/' + 
                   'composite_thickness_RGI60-all_regions/RGI60-XXXX/')
huss_fn_sample = 'RGI60-XXXX_thickness.tif'

if os.path.exists(input.ts_fp) == False:
    os.makedirs(input.ts_fp)
    
outdir_csv = input.outdir_emvel_fp 
outdir_fig = input.outdir_emvel_fp  + '../figures/'

if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)
if os.path.exists(outdir_fig) == False:
    os.makedirs(outdir_fig)



/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/HMA_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(input.mb_binned_fp_wdebris):
    if i.endswith('mb_bins_wdebris.csv'):
        rgiid_list.append(i[0:8])
        rgiid_fn_list.append(i)
        
rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)

print(len(rgiid_list))

main_glac_rgi = selectglaciersrgitable(rgiid_list)
main_glac_rgi['bin_fn'] = rgiid_fn_list

7934
4066 glaciers in region 13 are included in this model run: ['00093', '00130', '00135', '00137', '00140', '00147', '00175', '00181', '00183', '00203', '00210', '00277', '00358', '00382', '00386', '00391', '00394', '00400', '00401', '00403', '00439', '00440', '00441', '00465', '00561', '00585', '00594', '00604', '00606', '00611', '00628', '00643', '00693', '00713', '00750', '00751', '00757', '00761', '00763', '00777', '00788', '00809', '00830', '00834', '00838', '00880', '00884', '00885', '00891', '00905'] and more
2465 glaciers in region 14 are included in this model run: ['00005', '00018', '00032', '00036', '00043', '00057', '00063', '00072', '00088', '00101', '00104', '00111', '00131', '00142', '00145', '00146', '00159', '00163', '00164', '00187', '00213', '00219', '00222', '00225', '00235', '00243', '00251', '00271', '00287', '00309', '00323', '00326', '00336', '00346', '00347', '00352', '00353', '00363', '00366', '00367', '00370', '00372', '00380', '00398', '00403', '00432', '0

In [5]:
# Group datasets by nearest lat/lon
ds = xr.open_dataset(input.debris_elevstats_fullfn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi['CenLon'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))

latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))

main_glac_rgi['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi['latlon_unique_no'] = main_glac_rgi['latlon_nearidx'].map(latlon_unique_dict)

ds_latlon = ds.copy()

In [6]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
year_std = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
year_med = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
year_mad = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
doy_mean = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
doy_std = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
doy_med = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
doy_mad = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
dayfrac_mean = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
dayfrac_std = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
dayfrac_med = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
dayfrac_mad = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))

# for nlatlon, latlon_idx in enumerate([1095]):
for nlatlon, latlon_idx in enumerate(list(np.arange(0,len(latlon_nearidx_unique)))):
        
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['latlon_unique_no'] == latlon_idx]
    
    lat_idx, lon_idx = latlon_unique_dict_reversed[latlon_idx]
    
    print('\n', latlon_idx, ds_latlon['latitude'][lat_idx].values, ds_latlon['longitude'][lon_idx].values)

    df_all = None
    
    doy_list = []
    year_list = []
    dayfrac_list = []
        
        # =====
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[1]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if verbose:
            print(nglac, glac_idx, rgiid,'\n')
        
        df = pd.read_csv(input.mb_binned_fp_wdebris + main_glac_rgi_subset.loc[glac_idx,'bin_fn'])


        # Process only glaciers with debris
        debris_switch=False
        if ' perc_debris' in df.columns:
            # Process dataframe
            output_cns = ['# bin_center_elev_m', ' z1_bin_area_valid_km2', ' perc_debris']
            df = df[output_cns]
            df['# bin_center_elev_m'] = df['# bin_center_elev_m'].astype(np.float) 
            df[' z1_bin_area_valid_km2'] = df[' z1_bin_area_valid_km2'].astype(np.float)
            
            # Remove nan values
            df[' perc_debris'] = df[' perc_debris'].astype(np.float)
            df.fillna(0, inplace=True)
            
            if verbose:
                print(glac_str, df[' perc_debris'].max())
            
            df['area_debris_km2'] = df[' z1_bin_area_valid_km2'] * df[' perc_debris'] / 100
            
            if df[' perc_debris'].max() > 10:
                debris_switch = True
                debris_idx = np.where(df[' perc_debris'] > 50)[0]
                
        if debris_switch:
            if verbose:
                print('processing', glac_str)

            # ===== Project shapefile =====
            huss_dir = huss_dir_sample.replace('XXXX',str(region.zfill(2)))
            huss_fn = huss_fn_sample.replace('XXXX',glac_str)

            proj_fn = os.path.join(huss_dir, huss_fn) # THIS PROJECTION IS KEY!
            ds = gdal.Open(proj_fn)
            prj = ds.GetProjection()
            srs = osr.SpatialReference(wkt=prj)
            aea_srs = srs

            # If projected shapefile already exists, then skip projection
            glac_shp_proj_fn = glac_shp_proj_fp + glac_str + '_crs' + str(aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp'

            if os.path.exists(glac_shp_proj_fn) == False:
                glac_shp_proj = glac_shp_single.to_crs({'init': 'epsg:' + str(aea_srs.GetAttrValue("AUTHORITY", 1))})
                glac_shp_proj.to_file(glac_shp_proj_fn)

                # Shape layer processing
                glac_shp_init = gpd.read_file(glac_shp_fn_dict[region])
                if verbose:
                    print('Shp init crs:', glac_shp_init.crs)

                glac_shp_single = glac_shp_init[glac_shp_init['RGIId'] == rgiid]
                glac_shp_single = glac_shp_single.reset_index()


            glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
            glac_shp_lyr = glac_shp_ds.GetLayer()
            #This should be contained in features
            glac_shp_srs = glac_shp_lyr.GetSpatialRef()
            feat_count = glac_shp_lyr.GetFeatureCount()
            if verbose:
                print("Input glacier polygon count: %i" % feat_count)

            # Load DEM
            z1_dir = z1_dir_sample.replace('XXXX',str(region.zfill(2)))
            z1_fn = z1_fn_sample.replace('XXXX',glac_str)
            z1_ds = gdal.Open(z1_dir + z1_fn)
            z1_int_geom = geolib.ds_geom_intersection([z1_ds, z1_ds], t_srs=glac_shp_srs)

            glacfeat_list = []
            glacname_fieldname = "Name"
            glacnum_fieldname = "RGIId"
            glacnum_fmt = '%08.5f'

            for n, feat in enumerate(glac_shp_lyr):
                gf = GlacFeat(feat, glacname_fieldname, glacnum_fieldname)
                if verbose:
                    print("%i of %i: %s" % (n+1, feat_count, gf.feat_fn))
                #NOTE: Input must be in projected coordinate system, ideally equal area
                #Should check this and reproject
                gf.geom_attributes(srs=aea_srs)
                glacfeat_list.append(gf)

            if verbose:
                print(gf.feat_fn)

            fn_dict = OrderedDict()
            #We at least want to warp the two input DEMs
            fn_dict['z1'] = os.path.join(z1_dir, z1_fn)

            if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
                if verbose:
                    print(gf.glacnum)

                # Ice thickness data
                ice_thick_fn = os.path.join(huss_dir, huss_fn)
                if os.path.exists(ice_thick_fn):
                    fn_dict['ice_thick'] = ice_thick_fn


                if os.path.exists(input.ts_fp + input.ts_fn_dict[input.roi]):
                    fn_dict['ts'] = input.ts_fp + input.ts_fn_dict[input.roi]
                    
                if os.path.exists(input.ts_fp + input.ts_dayfrac_fn_dict[input.roi]):
                    fn_dict['ts_dayfrac'] = input.ts_fp + input.ts_dayfrac_fn_dict[input.roi]
                if os.path.exists(input.ts_fp + input.ts_year_fn_dict[input.roi]):
                    fn_dict['ts_year'] = input.ts_fp + input.ts_year_fn_dict[input.roi]
                if os.path.exists(input.ts_fp + input.ts_doy_fn_dict[input.roi]):
                    fn_dict['ts_doy'] = input.ts_fp + input.ts_doy_fn_dict[input.roi]


            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=buff_dist)
            if verbose:
                print("Expanding extent")
                print(gf.glac_geom_extent)
                print(warp_extent)
                print(aea_srs)

            #Warp everything to common res/extent/proj
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=input.ts_stats_res, \
                    extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
                    r='cubic')

            ds_dict = dict(zip(fn_dict.keys(), ds_list))

            if verbose:
                print(ds_list)
                print(fn_dict.keys())

            #Prepare mask for all glaciers within buffered area, not just the current glacier polygon
            glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
            glac_shp_lyr = glac_shp_ds.GetLayer()

            #Get global glacier mask
            #Want this to be True over ALL glacier surfaces, not just the current polygon
            glac_shp_lyr_mask = geolib.lyr2mask(glac_shp_lyr, ds_dict['ice_thick'])

            #Create buffer around glacier polygon
            glac_geom_buff = gf.glac_geom.Buffer(buff_dist)
            #This is False over glacier polygon surface, True elsewhere - can be applied directly
            glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])

            # ds masks
            ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
            dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
            dems_mask = dem1.mask
            if verbose:
                print('list of datasets:', len(ds_list_masked), fn_dict.values())

            #Combine to identify ~1 km buffer around glacier polygon over static rock
            static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
            static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']))

                #Now apply glacier mask AND mask NaN values
                glac_geom_mask = np.ma.mask_or(glac_geom_mask, dems_mask)
                gf.z1 = np.ma.array(gf.z1, mask=glac_geom_mask)

                if verbose:
                    print('\n\n# z1 pixels:', gf.z1.count(), '\n')
                if gf.z1.count() == 0:
                    if verbose:
                        print("No z1 pixels")
            else:
                print("Unable to load z1 ds")

            # ===== ADD VARIOUS LAYERS TO gf =====
            if nlatlon + nglac == 0:
                print('\n\nHACK TO BYPASS VALID AREA\n\n')
            gf.valid_area_perc = 100

            if gf.valid_area_perc < (100. * min_valid_area_perc):
                if verbose:
                    print("Not enough valid pixels. %0.1f%% percent of glacier polygon area" % (gf.valid_area_perc))
            #     return None

            else:
                #Filter dz - throw out abs differences >150 m

                #Compute dz, volume change, mass balance and stats
                gf.z1_stats = malib.get_stats(gf.z1)
                z1_elev_med = gf.z1_stats[5]
                z1_elev_min, z1_elev_max = malib.calcperc(gf.z1, (0.1, 99.9))

                #Caluclate stats for aspect and slope using z2
                #Requires GDAL 2.1+
                gf.z1_aspect = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='aspect', returnma=True), mask=glac_geom_mask)
                gf.z1_aspect_stats = malib.get_stats(gf.z1_aspect)
                z1_aspect_med = gf.z1_aspect_stats[5]
                gf.z1_slope = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='slope', returnma=True), mask=glac_geom_mask)
                gf.z1_slope_stats = malib.get_stats(gf.z1_slope)
                z1_slope_med = gf.z1_slope_stats[5]

                #Can estimate ELA values computed from hypsometry and typical AAR
                #For now, assume ELA is mean
                gf.z1_ela = None
                gf.z1_ela = gf.z1_stats[3]
                #Note: in theory, the ELA should get higher with mass loss
                #In practice, using mean and same polygon, ELA gets lower as glacier surface thins


                if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
                    if 'ice_thick' in ds_dict:
                        #Load ice thickness
                        gf.H = np.ma.array(iolib.ds_getma(ds_dict['ice_thick']), mask=glac_geom_mask)
                        gf.H_mean = gf.H.mean()
                        if verbose:
                            print('mean ice thickness [m]:', gf.H_mean)

                    if 'ts' in ds_dict:
                        #Load surface temperature maps
                        gf.ts = np.ma.array(iolib.ds_getma(ds_dict['ts']), mask=glac_geom_mask)
                        gf.ts.mask = np.ma.mask_or(glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    else:
                        gf.ts = None
                        
                    if 'ts_dayfrac' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_dayfrac = np.ma.array(iolib.ds_getma(ds_dict['ts_dayfrac']), mask=glac_geom_mask)
                        gf.ts_dayfrac.mask = np.ma.mask_or(glac_geom_mask, 
                                                           np.ma.getmask(np.ma.masked_array(gf.ts_dayfrac.data, np.isnan(gf.ts_dayfrac.data))))
                    else:
                        gf.ts_dayfrac = None
                        
                    if 'ts_year' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_year = np.ma.array(iolib.ds_getma(ds_dict['ts_year']), mask=glac_geom_mask)
                        gf.ts_year.mask = np.ma.mask_or(glac_geom_mask, 
                                                        np.ma.getmask(np.ma.masked_array(gf.ts_year.data, np.isnan(gf.ts_year.data))))
                    else:
                        gf.ts_year = None
                        
                    if 'ts_doy' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_doy = np.ma.array(iolib.ds_getma(ds_dict['ts_doy']), mask=glac_geom_mask)
                        gf.ts_doy.mask = np.ma.mask_or(glac_geom_mask, 
                                                       np.ma.getmask(np.ma.masked_array(gf.ts_doy.data, np.isnan(gf.ts_doy.data))))
                    else:
                        gf.ts_doy = None

                gf.res = geolib.get_res(ds_dict['z1'])

                if verbose:
                    print('Area [km2]:', gf.glac_area / 1e6)
                    print('-------------------------------')
                    
                # Isolate values with positive surface temperatures below mean elevation
                zmean_mask = np.ma.mask_or(glac_geom_mask,  
                                           np.ma.getmask(np.ma.masked_greater(gf.z1, gf.z1.compressed().mean())))
                ts_zmean_mask = np.ma.mask_or(zmean_mask,
                                              np.ma.getmask(np.ma.masked_less(gf.ts, 0)))

                gf.ts_doy.mask = zmean_mask
                gf.ts_year.mask = zmean_mask
                gf.ts_dayfrac.mask = zmean_mask
                doy_list.extend(list(gf.ts_doy.compressed()))
                year_list.extend(list(gf.ts_year.compressed()))
                dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))
                
    
    # Compute statistics
    year_mean_latlon = np.mean(year_list)
    year_std_latlon = np.std(year_list)
    year_med_latlon = malib.fast_median(year_list)
    year_mad_latlon = malib.mad(year_list)
    doy_mean_latlon = np.mean(doy_list)
    doy_std_latlon = np.std(doy_list)
    doy_med_latlon = malib.fast_median(doy_list)
    doy_mad_latlon = malib.mad(doy_list)
    dayfrac_mean_latlon = np.mean(dayfrac_list)
    dayfrac_std_latlon = np.std(dayfrac_list)
    dayfrac_med_latlon = malib.fast_median(dayfrac_list)
    dayfrac_mad_latlon = malib.mad(dayfrac_list)
    
    # Update array
    year_mean[lat_idx,lon_idx] = year_mean_latlon
    year_std[lat_idx,lon_idx] = year_std_latlon
    year_med[lat_idx,lon_idx] = year_med_latlon
    year_mad[lat_idx,lon_idx] = year_mad_latlon
    doy_mean[lat_idx,lon_idx] = doy_mean_latlon
    doy_std[lat_idx,lon_idx] = doy_std_latlon
    doy_med[lat_idx,lon_idx] = doy_med_latlon
    doy_mad[lat_idx,lon_idx] = doy_mad_latlon
    dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
    dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
    dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
    dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon
    
#     print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
#     print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
#     print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
#     print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))   
    
    


 0 45.0 79.25


HACK TO BYPASS VALID AREA



 1 45.0 79.5

 2 45.0 79.75

 3 45.0 80.0

 4 45.0 80.25

 5 45.0 80.5

 6 44.75 79.75

 7 44.75 80.0

 8 44.75 80.5

 9 44.75 80.75

 10 44.5 80.25

 11 44.5 80.5

 12 44.25 83.25

 13 44.25 83.5

 14 44.0 83.25

 15 44.0 83.5

 16 44.0 83.75

 17 44.0 84.0

 18 43.75 84.0

 19 43.75 84.25

 20 43.75 84.5

 21 43.75 84.75

 22 43.75 85.0

 23 43.75 85.25

 24 43.75 85.75

 25 43.75 88.25

 26 43.75 88.5

 27 43.5 84.5

 28 43.5 84.75

 29 43.5 85.0

 30 43.5 85.25

 31 43.5 85.5

 32 43.5 85.75

 33 43.5 86.0

 34 43.5 89.25

 35 43.5 93.25

 36 43.25 77.5

 37 43.25 83.75

 38 43.25 85.0

 39 43.25 85.5

 40 43.25 85.75

 41 43.25 94.25

 42 43.0 76.75

 43 43.0 77.0

 44 43.0 77.25

 45 43.0 77.5

 46 43.0 77.75

 47 43.0 82.75

 48 43.0 83.5

 49 43.0 83.75

 50 43.0 87.0

 51 43.0 94.25

 52 43.0 94.5

 53 42.75 76.75

 54 42.75 77.0

 55 42.75 77.25

 56 42.75 81.25

 57 42.75 81.75

 58 42.75 82.25

 59 42.75 82.5

 60 42.75 82.75

 

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty


 85 42.5 85.25

 86 42.5 85.5

 87 42.25 70.75

 88 42.25 71.0

 89 42.25 71.25

 90 42.25 74.75

 91 42.25 78.25

 92 42.25 78.5

 93 42.25 78.75

 94 42.25 79.0

 95 42.25 79.25

 96 42.25 79.5

 97 42.25 79.75

 98 42.25 80.0

 99 42.25 80.25

 100 42.25 80.5

 101 42.25 80.75

 102 42.25 81.0

 103 42.25 81.25

 104 42.25 81.5

 105 42.25 81.75

 106 42.0 70.5

 107 42.0 71.0

 108 42.0 72.0

 109 42.0 72.25

 110 42.0 76.0

 111 42.0 76.5

 112 42.0 76.75

 113 42.0 77.0

 114 42.0 77.25

 115 42.0 77.5

 116 42.0 77.75

 117 42.0 78.0

 118 42.0 78.25

 119 42.0 78.5

 120 42.0 78.75

 121 42.0 79.0

 122 42.0 79.25

 123 42.0 79.5

 124 42.0 79.75

 125 42.0 80.0

 126 42.0 80.25

 127 42.0 80.5

 128 42.0 80.75

 129 42.0 81.0

 130 41.75 76.25

 131 41.75 77.25

 132 41.75 77.5

 133 41.75 77.75

 134 41.75 78.0

 135 41.75 78.25

 136 41.75 78.5

 137 41.75 78.75

 138 41.75 79.0

 139 41.75 80.0

 140 41.75 80.25

 141 41.75 80.5

 142 41.5 77.25

 143 41.5 77.5

 144 41.5 


 567 35.75 93.25

 568 35.75 93.5

 569 35.75 93.75

 570 35.75 94.25

 571 35.5 70.75

 572 35.5 72.5

 573 35.5 72.75

 574 35.5 73.0

 575 35.5 74.75

 576 35.5 75.0

 577 35.5 75.25

 578 35.5 75.5

 579 35.5 75.75

 580 35.5 76.0

 581 35.5 76.25

 582 35.5 76.5

 583 35.5 76.75

 584 35.5 77.0

 585 35.5 77.25

 586 35.5 77.5

 587 35.5 77.75

 588 35.5 78.0

 589 35.5 78.5

 590 35.5 78.75

 591 35.5 80.0

 592 35.5 80.5

 593 35.5 80.75

 594 35.5 81.0

 595 35.5 81.25

 596 35.5 81.5

 597 35.5 81.75

 598 35.5 82.25

 599 35.5 89.5

 600 35.5 89.75

 601 35.5 94.5

 602 35.25 72.75

 603 35.25 73.0

 604 35.25 73.5

 605 35.25 74.5

 606 35.25 74.75

 607 35.25 75.0

 608 35.25 75.25

 609 35.25 76.25

 610 35.25 76.5

 611 35.25 76.75

 612 35.25 77.0

 613 35.25 77.25

 614 35.25 77.5

 615 35.25 77.75

 616 35.25 78.0

 617 35.25 78.25

 618 35.25 78.5

 619 35.25 78.75

 620 35.25 79.0

 621 35.25 80.5

 622 35.25 80.75

 623 35.25 81.0

 624 35.25 81.25

 625 35.25 81.5


 1046 28.75 90.25

 1047 28.75 93.25

 1048 28.75 93.5

 1049 28.75 96.5

 1050 28.75 97.75

 1051 28.75 98.25

 1052 28.5 83.75

 1053 28.5 84.0

 1054 28.5 84.25

 1055 28.5 84.5

 1056 28.5 84.75

 1057 28.5 85.0

 1058 28.5 85.25

 1059 28.5 85.5

 1060 28.5 85.75

 1061 28.5 91.0

 1062 28.5 91.25

 1063 28.5 96.5

 1064 28.5 97.5

 1065 28.5 98.25

 1066 28.5 98.5

 1067 28.5 98.75

 1068 28.25 85.0

 1069 28.25 85.25

 1070 28.25 85.5

 1071 28.25 85.75

 1072 28.25 86.0

 1073 28.25 86.25

 1074 28.25 86.5

 1075 28.25 86.75

 1076 28.25 87.0

 1077 28.25 87.5

 1078 28.25 89.5

 1079 28.25 89.75

 1080 28.25 90.0

 1081 28.25 90.25

 1082 28.25 90.5

 1083 28.25 90.75

 1084 28.25 91.0

 1085 28.25 91.25

 1086 28.25 91.5

 1087 28.25 91.75

 1088 28.25 92.75

 1089 28.25 97.0

 1090 28.25 97.5

 1091 28.25 98.75

 1092 28.0 86.0

 1093 28.0 86.25

 1094 28.0 86.5

 1095 28.0 86.75

 1096 28.0 87.0

 1097 28.0 87.25

 1098 28.0 87.5

 1099 28.0 87.75

 1100 28.0 88.0

 1101 2

In [7]:
print('DONE')

DONE


In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': ds_latlon.latitude.values,
                                  'longitude': ds_latlon.longitude.values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 81, longitude: 161)
Coordinates:
  * latitude      (latitude) float32 45.0 44.75 44.5 44.25 ... 25.5 25.25 25.0
  * longitude     (longitude) float32 65.0 65.25 65.5 ... 104.5 104.75 105.0
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude, 

In [9]:
lat_idx = 68
lon_idx = 88
# lat_idx = 37
# lon_idx = 46
print(ds_latlon['latitude'][lat_idx].values, ds_latlon['longitude'][lon_idx].values,
      '\n', ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
      ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

28.0 87.0 
 2015.0001220703125 0.0016057980246841908 229.2476348876953 51.61998748779297


In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/HMA_debris_tsinfo.nc'